## Web Scraping • Private APIs • Selenium • JSON • Pandas • Data Cleaning

---

## 1️⃣ WEBSITE ACCESS & HTTP REQUESTS

### 🔹 Library
```python
import requests
Explanation
requests is a Python library used to send HTTP requests (GET, POST, etc.) and receive responses from web servers.

🔹 Core method
requests.get(url, headers=headers)
Explanation

Sends an HTTP GET request to the server

Returns a Response object containing status code, headers, and data

🔹 Why headers are required
Avoid 403 Forbidden

Mimic real browser behavior

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
}
Explanation
Many websites block requests that don’t look like they come from a real browser.
User-Agent tells the server who you are pretending to be.

2️⃣ BEAUTIFULSOUP (HTML PARSING)
🔹 Correct import
from bs4 import BeautifulSoup
❌ Wrong

import bs4 as BeautifulSoup
Explanation

BeautifulSoup is a class, not a module

Wrong import causes: TypeError: 'module' object is not callable

🔹 Create soup object
soup = BeautifulSoup(response.text, "lxml")
Explanation

response.text → raw HTML

"lxml" → fast HTML parser

🔹 Errors encountered & meaning
Error	Meaning
module not callable	Wrong import
FeatureNotFound: lxml	Parser not installed
Akamai error page	Bot protection
📌 Conclusion:
HTML scraping alone is unreliable for modern, JS-heavy websites.

3️⃣ NETWORK TAB → API DISCOVERY
🔹 Tools
Chrome DevTools → Network

Filter: Fetch / XHR

🔹 What you inspected
Request URL

Headers

Cookies

Payload

HTTP Method (POST)

🔹 Result
You discovered a private JSON API endpoint:

/servicegateway-ambitionbox/company-services/...
Explanation
Modern websites load data via APIs, not HTML.
Network tab lets you reverse-engineer those APIs.

4️⃣ WHY DIRECT API CALLS FAIL (403)
🔹 Cause
Missing:

Cookies

System headers

Browser session

🔹 Why pd.read_json(url) failed
It cannot send:

Headers

POST payload

Cookies

Explanation
pd.read_json() only works for open public APIs, not protected ones.

5️⃣ SELENIUM — WHY & WHEN IT IS USED
🔹 Why Selenium is needed
Execute JavaScript

Create a trusted browser session

Generate valid cookies

Bypass bot detection

🔹 Library
from selenium import webdriver
Explanation
Selenium controls a real browser, so the website trusts it.

6️⃣ SELENIUM WORKFLOW (ALGORITHM)
Launch browser

Open target website

Let JavaScript load

Extract cookies

Transfer cookies to requests.Session

Call private API

7️⃣ SELENIUM CODE (CORE CONCEPTS)
🔹 Launch browser
driver = webdriver.Chrome()
driver.get("https://www.ambitionbox.com/list-of-companies?page=1")
Explanation

Opens Chrome

Loads JS

Generates session cookies

🔹 Extract cookies
cookies = driver.get_cookies()
Each cookie is a dictionary:

{
  'name': 'PHPSESSID',
  'value': 'abc123',
  'domain': '.ambitionbox.com'
}
Explanation
Cookies prove to the server that your request comes from a valid browser session.

8️⃣ TRANSFERRING COOKIES TO REQUESTS
🔹 Create session
session = requests.Session()
Explanation
A Session persists:

Cookies

Headers

Authentication

🔹 Add cookies
for cookie in cookies:
    session.cookies.set(cookie['name'], cookie['value'])
Explanation
Transfers Selenium’s browser identity into requests.

🔹 Set headers
session.headers.update({
    "User-Agent": "Mozilla/5.0",
    "Referer": "https://www.ambitionbox.com/"
})
📌 Result:
Requests now behaves exactly like Selenium’s browser.

9️⃣ CALLING PRIVATE API (POST REQUEST)
payload = {
    "page": 1,
    "sortBy": "popular",
    "limit": 20
}

response = session.post(api_url, json=payload)
data = response.json()
Explanation

post() → sends data to server

json=payload → auto JSON encoding

.json() → converts response into Python dict/list

🔟 JSON STRUCTURE ANALYSIS
🔹 Inspect JSON
data.keys()
type(data)
Explanation
Always inspect structure before indexing.

🔹 Common mistake
data["cards"]["companies"]  # ❌
Why wrong?
cards is a list, not a dictionary.

🔹 Correct approach
type(data["cards"])
data["cards"][0].keys()
Key lesson:
👉 Lists → index with numbers
👉 Dicts → index with keys

1️⃣1️⃣ PANDAS DATAFRAME CREATION
🔹 Create DataFrame
df = pd.DataFrame(companies)
Explanation

Converts list of dictionaries into tabular form

Keys → columns

🔹 Inspect
df.head()
df.info()
df.columns
Method	Purpose
head()	Preview rows
info()	Data types & nulls
columns	Column names
1️⃣2️⃣ ACCESSING NESTED DATA IN COLUMNS
🔹 Safe extraction pattern
df['column'] = df['column'].apply(
    lambda x: x.get('name') if isinstance(x, dict) else None
)
🔹 Concept explanations
apply() → applies function to each row

lambda → anonymous function

isinstance() → type safety

.get() → avoids KeyError

1️⃣3️⃣ DEALING WITH NUMERIC None VALUES FROM API
🔹 Convert safely
df['col'] = pd.to_numeric(df['col'], errors='coerce')
Explanation

Converts invalid values to NaN

Prevents crashes

🔹 Handling strategies
Data Type	Strategy
Counts	Fill with 0
Ratings	Median
Salaries	Median or keep NaN
ML	Missing-value flag
df['minCtc_missing'] = df['minCtc'].isna().astype(int)
1️⃣4️⃣ IMPORTANT METHODS & FUNCTIONS USED
🧰 Python
type() → object type

isinstance() → type checking

len() → length

iter() / next() → iteration

🧰 Requests
get() → fetch data

post() → send data

Session() → persistent state

headers.update() → browser identity

🧰 Selenium
webdriver.Chrome() → browser

driver.get() → load page

driver.get_cookies() → extract session

🧰 Pandas
DataFrame() → table creation

apply() → row-wise logic

fillna() → missing values

astype() → type conversion

to_numeric() → safe numeric casting

isna() → missing detection

head() / info() → inspection

🧠 FINAL PIPELINE (MENTAL MODEL)
Website
   ↓
Selenium (browser session)
   ↓
Cookies + Headers
   ↓
Requests Session
   ↓
Private API (JSON)
   ↓
Pandas DataFrame
   ↓
Cleaning & Transformation
   ↓
ML / Analysis Ready Data
🎯 WHAT YOU ACTUALLY LEARNED (BIG PICTURE)
✔ Modern scraping ≠ HTML
✔ APIs are hidden but discoverable
✔ Selenium is for session trust, not scraping
✔ JSON ≠ always dict (lists matter!)
✔ Pandas requires defensive coding
✔ Missing data must be treated semantically